### Importing all libraries/modules required

In [221]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import io
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import folium

### Fetching the location of Mumbai and Ratnagiri

In [222]:
mumbai = 'Mumbai'
ratnagiri = 'Ratnagiri'

geolocator = Nominatim(user_agent="foursquare_agent")
mum_location = geolocator.geocode(mumbai)
mum_latitude = mum_location.latitude
mum_longitude = mum_location.longitude

rat_location = geolocator.geocode(ratnagiri)
rat_latitude = rat_location.latitude
rat_longitude = rat_location.longitude

print("Location of Mumbai : ", mum_latitude, mum_longitude)
print("Location of Ratnagiri : ", rat_latitude, rat_longitude)

Location of Mumbai :  18.9387711 72.8353355
Location of Ratnagiri :  17.282607900000002 73.4569787039826


### Setting up the credentials for Forsquare API and Creating the url for Mumbai Hospitals and Ratnagiri Hospitals

In [240]:
CLIENT_ID = 'O4KNNRFK0AAZXWP4DBZPEDQPQ2PYL0AYLAY2SOAFZCIPR352' # your Foursquare ID
CLIENT_SECRET = 'BYQ4EVBZ4EUIGNIP5ZEQIWASGZ2JWU5NLH0RPZ44LYDE0IHX' # your Foursquare Secret
VERSION = '20180604'
search_query = 'hospital'
LIMIT = 100000
radius = 6000000000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

mum_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, mum_latitude, mum_longitude, VERSION, search_query, radius, LIMIT)
rat_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, rat_latitude, rat_longitude, VERSION, search_query, radius, LIMIT)


print("\nMumbai url :", mum_url)
print("Ratnagiri url :", rat_url)


Your credentails:
CLIENT_ID: O4KNNRFK0AAZXWP4DBZPEDQPQ2PYL0AYLAY2SOAFZCIPR352
CLIENT_SECRET:BYQ4EVBZ4EUIGNIP5ZEQIWASGZ2JWU5NLH0RPZ44LYDE0IHX

Mumbai url : https://api.foursquare.com/v2/venues/search?client_id=O4KNNRFK0AAZXWP4DBZPEDQPQ2PYL0AYLAY2SOAFZCIPR352&client_secret=BYQ4EVBZ4EUIGNIP5ZEQIWASGZ2JWU5NLH0RPZ44LYDE0IHX&ll=18.9387711,72.8353355&v=20180604&query=hospital&radius=6000000000&limit=100000
Ratnagiri url : https://api.foursquare.com/v2/venues/search?client_id=O4KNNRFK0AAZXWP4DBZPEDQPQ2PYL0AYLAY2SOAFZCIPR352&client_secret=BYQ4EVBZ4EUIGNIP5ZEQIWASGZ2JWU5NLH0RPZ44LYDE0IHX&ll=17.282607900000002,73.4569787039826&v=20180604&query=hospital&radius=6000000000&limit=100000


### Fetching the result from foursqare API using the urls created

In [241]:
mum_results = requests.get(mum_url).json()
rat_results = requests.get(rat_url).json()

df_mum = json_normalize(mum_results['response']['venues'])
df_rat = json_normalize(rat_results['response']['venues'])

### EDA and Data Wranggling

In [242]:
df_mum.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4ea13b8277c8d0ce5d6aba4e,Bombay Hospital & Medical Research Centre,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1588623945,False,"12, Marine Lines",18.940915,72.828326,"[{'label': 'display', 'lat': 18.94091475484561...",775,400 020,IN,Mumbai,Mahārāshtra,India,"[12, Marine Lines, Mumbai 400 020, Mahārāshtra...",NaN
1,4c0f24891281c9b6c4361e1d,St. George Hospital/college,[],v-1588623945,False,D.N. road,18.939797,72.834989,"[{'label': 'display', 'lat': 18.93979668617248...",119,NaN,IN,NaN,Mahārāshtra,India,"[D.N. road, Mahārāshtra, India]",NaN
2,4f96ab78e4b0f19802fb8033,St. George Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1588623945,False,CST,18.937614,72.837459,"[{'label': 'display', 'lat': 18.937614, 'lng':...",258,NaN,IN,Mumbai,Mahārāshtra,India,"[CST, Mumbai, Mahārāshtra, India]",NaN
3,4ec6612abe7ba4fc6d11d996,Cama & Albless Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1588623945,False,NaN,18.941590,72.832684,"[{'label': 'display', 'lat': 18.94159016459993...",420,NaN,IN,NaN,NaN,India,[India],NaN
4,53e05e23498ee2ab1ea3565b,Seaman Hospital,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1588623945,False,NaN,18.936524,72.839380,"[{'label': 'display', 'lat': 18.936524, 'lng':...",493,NaN,IN,NaN,NaN,India,[India],NaN


In [243]:
df_rat.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,500bdc83e4b0092312c09c10,Nayb Eye Hospital,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",v-1588623946,False,Near Juna Bhairi,SH78,17.529335,73.519798,"[{'label': 'display', 'lat': 17.52933502197265...",28264,415605,IN,Chiplūn,Mahārāshtra,India,"[Near Juna Bhairi (SH78), Chiplūn 415605, Mahā...",NaN
1,4edb49318b8173160d66e788,Dr.kanitkar Hospital,"[{'id': '4bf58dd8d48988d177941735', 'name': 'D...",v-1588623946,False,NaN,NaN,17.054340,73.634834,"[{'label': 'display', 'lat': 17.05434036254882...",31678,NaN,IN,NaN,NaN,India,[India],NaN
2,51f0e559498e05fb7c0f7e1c,Muktangan Accident Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1588623946,False,NaN,NaN,17.691280,74.002271,"[{'label': 'display', 'lat': 17.69127969228998...",73631,NaN,IN,NaN,NaN,India,[India],NaN
3,53083e9211d2180166a088a9,Dr.Pendharkar Memorial Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1588623946,False,NaN,NaN,17.686102,74.008520,"[{'label': 'display', 'lat': 17.68610188196328...",73802,NaN,IN,NaN,NaN,India,[India],NaN
4,520b322d11d26a071005f0c2,Satara Civil Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1588623946,False,NaN,NaN,17.689874,74.006730,"[{'label': 'display', 'lat': 17.68987416018504...",73907,NaN,IN,NaN,NaN,India,[India],NaN


In [244]:
df_mum.describe()

,location.lat,location.lng,location.distance
count,50.000000,50.000000,50.000000
mean,18.962224,72.828730,3081.400000
std,0.023598,0.012163,2509.409094
min,18.926430,72.795690,119.000000
25%,18.951537,72.820344,1686.500000
50%,18.958781,72.830307,2725.000000
75%,18.964492,72.835963,3380.000000
max,19.040060,72.864769,11693.000000


In [245]:
df_rat.describe()

,location.lat,location.lng,location.distance
count,50.000000,50.000000,50.000000
mean,17.337645,74.105946,96441.120000
std,0.606970,0.239301,24925.677055
min,16.684834,73.510058,28264.000000
25%,16.717694,73.997781,76683.250000
50%,17.260770,74.179089,104221.000000
75%,17.690928,74.236819,105858.000000
max,18.464380,74.593018,136934.000000


### Dropping the unwanted/undesired columns

In [246]:
df_mum = df_mum[['name', 'location.lat', 'location.lng', 'location.distance']]
df_mum.rename(columns={'location.lat':'lat', 'location.lng':'lng', 'location.distance':'distance'}, inplace=True)
df_mum.head()

,name,lat,lng,distance
0,Bombay Hospital & Medical Research Centre,18.940915,72.828326,775
1,St. George Hospital/college,18.939797,72.834989,119
2,St. George Hospital,18.937614,72.837459,258
3,Cama & Albless Hospital,18.941590,72.832684,420
4,Seaman Hospital,18.936524,72.839380,493


In [247]:
df_rat = df_rat[['name', 'location.lat', 'location.lng', 'location.distance']]
df_rat.rename(columns={'location.lat':'lat', 'location.lng':'lng', 'location.distance':'distance'}, inplace=True)
df_rat.head()

,name,lat,lng,distance
0,Nayb Eye Hospital,17.529335,73.519798,28264
1,Dr.kanitkar Hospital,17.054340,73.634834,31678
2,Muktangan Accident Hospital,17.691280,74.002271,73631
3,Dr.Pendharkar Memorial Hospital,17.686102,74.008520,73802
4,Satara Civil Hospital,17.689874,74.006730,73907


## Mapping the hosiptals on folium map and placing marlers

### Mumbai

In [248]:
mum_map = folium.Map(location=[mum_latitude, mum_longitude], zoom_start=13)

folium.CircleMarker(
    [mum_latitude, mum_longitude],
    radius=10,
    color='red',
    popup=mumbai,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(mum_map)


for lat, lng, label in zip(df_mum.lat, df_mum.lng, df_mum.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(mum_map)

mum_map

### Ratnagiri

In [249]:
rat_map = folium.Map(location=[rat_latitude, rat_longitude], zoom_start=10)

folium.CircleMarker(
    [rat_latitude, rat_longitude],
    radius=10,
    color='red',
    popup=ratnagiri,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(rat_map)


for lat, lng, label in zip(df_rat.lat, df_rat.lng, df_rat.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(rat_map)

rat_map

In [233]:
print("Closest Hospital to Mumbai : %d Mts"% df_mum['distance'].min())
print("Closest Hospital to Ratnagiri : %d Mts"% df_rat['distance'].min())

Closest Hospital to Mumbai : 119 Mts
Closest Hospital to Ratnagiri : 28264 Mts
